# Extend
Like its parent, `ipykernel`, `irobotframework` is extensible via a number of mechanisms.

## Magics

`irobotframework` ships with two magic functions, `%%python module` and `%%tidy`.

Any python module, such as a test library, can extend these behaviors with `register_robot_cell_magic`.

In [ ]:
%%python module MagicShout
from irobotframework import register_robot_cell_magic

def magic_shout(code, **kwargs):
    return dict(
        status="ok", payload=[
            dict(source="set_next_input", 
                 text="\n".join(code.upper().split("\n")[1:]), 
                 replace=True)
        ]
    )

register_robot_cell_magic("SHOUT", r"%%SHOUT", magic_shout)

Now, executing a cell that starts with `%%SHOUT` will update the cell to be all upper case. The above example returns a raw [execution result](https://jupyter-client.readthedocs.io/en/stable/messaging.html#execute), and so the code won't actually be executed. Returning a string will execute the code as normal, but not change the code.
```robotframework
%%SHOUT
*** tasks ***
unrelenting force
    log  fus ro dah
```
Would yield the following:
```robotframework
*** TASKS ***
UNRELENTING FORCE
    LOG  FUS RO DAH
```

## Unmagic
If, for some reason, you'd like to remove a magic, you can do so with `unregister_robot_cell_magic`:

In [ ]:
%%python module Shush
from irobotframework import unregister_robot_cell_magic

unregister_robot_cell_magic(r"SHOUT")

Now, you can't `%%SHOUT` anymore.

## Completions

Code completion is provided for a number of Robot Framework features:

- magics
- table names
- importable libraries
- suite and test/task settings
- locally-defined variables
- keywords

These can be extended, again at the python level, with `register_robot_completion_finder`, which return a tuple of a simple list for more basic implementations and the `_jupyter_types_experimental` type:

In [ ]:
%%python module WuFinder
# typing for good luck
from typing import List, Tuple

from irobotframework.completer import Completer
from irobotframework import register_robot_completion_finder

def complete_wu_tang_discography(
    completer: Completer,
    line: str,
    code: str,
    cursor_pos: int,
    line_cursor: int,
    offset: int,
    history: List[str],
) -> Tuple[List[str], List[dict]]:
    if line.strip().endswith("Wu"):
        matches = [
            f"{line[:line_cursor-2]}{album}"
            for album in [
                "Enter the Wu-Tang (36 Chambers)",
                "Wu-Tang Forever",
                "The W",
                "Iron Flag",
                "8 Diagrams",
                "A Better Tomorrow",
                "Once Upon a Time in Shaolin"
            ]
        ]
        return matches, [
            {
                "start": cursor_pos,
                "end": offset + len(line),
                "type": "wu-tang album",
                "text": m,
            }
            for m in matches
        ],
    return [], []
    

register_robot_completion_finder(complete_wu_tang_discography)

Now, pressing <kbd>TAB</kbd> on a line that ends with `Wu` will give back a number of different options.

In [ ]:
*** Test Case **
Check out an album
    Log  Wu

### Prioritize
Additionally, you can pass a `before` or `after` completion finder (such as the defaults from `irobotframework.completion_finders`).

### Uncomplete
Once again, you can `register_robot_completion_finder`:

In [ ]:
%%python module WuFinder
from irobotframework import unregister_robot_completion_finder
import WuFinder
    
unregister_robot_completion_finder(WuFinder.complete_wu_tang_discography)

## Inspections

> TBD